# Experiment 1: Single stage suborbital launcher
This experiment contains a single stage lunar suborbital launcher whose goal is to reach the highest possible altitude. The launcher is vertical w.r.t. to the surface. The only actions the agent can take are to ignite and turnoff the engine.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [49]:
from dataclasses import dataclass
from math import radians
import time

import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import trange

from cw.context import time_it
from cw.simulation import Simulation, StatesBase, AB3Integrator, BatchLogger, Logging
from cw.filters import smooth_signal

from topone.sim_post_processing import sim_post_processing
from topone.environment import Environment, Stage
from topone.pid_agent import PIDAgent

from agent import Agent

## Simulation configuration

In [3]:
@dataclass
class States(StatesBase):
    t: float = 0
    command_engine_on: bool = False
    command_drop_stage: bool = False
    gii: np.ndarray = np.zeros(2)
    xii: np.ndarray = np.zeros(2)
    vii: np.ndarray = np.zeros(2)
    aii: np.ndarray = np.zeros(2)
    fii_thrust: np.ndarray = np.zeros(2)
    theta: float = 0.
    theta_dot: float = 0.
    mass: float = 0.
    mass_dot: float = 0.
    h: float = 0.
    engine_on: bool = False
    stage_state: int = 0
    stage_idx: int = 0
    gamma_i: float = 0.
    gamma_e: float = 0.
    latitude: float = 0.
        
    reward: float = 0.
    reward_integral: float = 0.
    delta_v: float = 0.

    def get_y_dot(self):
        y = np.empty(7)
        y[:2] = self.vii
        y[2:4] = self.aii
        y[4] = self.theta_dot
        y[5] = self.mass_dot
        y[6] = self.reward
        return y

    def get_y(self):
        y = np.empty(7)
        y[:2] = self.xii
        y[2:4] = self.vii
        y[4] = self.theta
        y[5] = self.mass
        y[6] = self.reward_integral
        return y

    def set_t_y(self, t, y):
        self.t = t
        self.xii = y[:2]
        self.vii = y[2:4]
        self.theta = y[4]
        self.mass = y[5]
        self.reward_integral = y[6]

In [65]:
agent = Agent(
    epsilon=0.1,
    alpha=0.9,
    gamma=0.7,
    path="./set_1"
)

simulation = Simulation(
    states_class=States,
    integrator=AB3Integrator(
        h=0.1,
        rk4=True,
        fd_max_order=1),
    modules=[
        Environment(
            surface_diameter=1737.4e3,
            mu=4.9048695e12,
            stages=(
                Stage(
                    dry_mass=1,
                    propellant_mass=0.02,
                    specific_impulse=100,
                    thrust=2*1.7),
            ),
            initial_altitude=1000,
            initial_theta_e=radians(90),
            initial_latitude=radians(90),
            reward_function=lambda s: s.h - 1000,
        ),
        agent
    ],
    logging=Logging(),
    initial_state_values=None,
)
batch_logger = BatchLogger()
batch_logger.initialize(simulation)
simulation.stash_states()

## Batch run

In [37]:
def run_batch(n_episodes, backup_period=30):
    batch_logger.reset_batch()
    
    # Backup original logger and swap with faster logger
    original_logger = simulation.logging
    simulation.logging = batch_logger
    
    last_backup_time = time.time()
    
    try:
        for i in trange(n_episodes):
            simulation.restore_states()
            result = simulation.run(100)
            if time.time() - last_backup_time >= backup_period:                
                agent.store()
                last_backup_time = time.time()
    except:
        raise
    finally:
        agent.store()
        simulation.logging = original_logger
        batch_results = batch_logger.finish_batch()        
        return batch_results

In [44]:
batch_results = run_batch(10000)
display(agent.values)
agent.display_greedy_policy()
display(batch_results)

array([[ -84.77655066,  -56.2606161 ],
       [-120.21637567,  -66.68648207],
       [ 107.46613404,  107.46168827]])

UNFIRED: True 
FIRING: True
FIRED: False


<xarray.Dataset>
Dimensions:             (d_2_0: 2, idx: 10000)
Coordinates:
  * idx                 (idx) int64 0 1 2 3 4 5 ... 9995 9996 9997 9998 9999
Dimensions without coordinates: d_2_0
Data variables:
    t                   (idx) float64 10.0 10.0 10.0 10.0 ... 10.0 10.0 10.0
    command_engine_on   (idx) bool False False False False ... False True False
    command_drop_stage  (idx) bool False False False False ... False False False
    gii                 (idx, d_2_0) float64 -9.939e-17 -1.623 ... -1.623
    xii                 (idx, d_2_0) float64 1.064e-10 1.738e+06 ... 1.738e+06
    vii                 (idx, d_2_0) float64 -9.749e-16 -15.92 ... -1.28
    aii                 (idx, d_2_0) float64 -9.939e-17 -1.623 ... -1.623
    fii_thrust          (idx, d_2_0) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    theta               (idx) float64 1.571 1.571 1.571 ... 1.571 1.571 1.571
    theta_dot           (idx) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    mass                (idx) float64 1.1 1.096 1.099 ... 1.098 1.098 1.083
    mass_dot            (idx) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    h                   (idx) float64 923.5 950.9 929.5 ... 938.3 1.032e+03
    engine_on           (idx) bool False False False False ... False False False
    stage_state         (idx) int64 2 2 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2 2 2 2
    stage_idx           (idx) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    gamma_i             (idx) float64 -1.571 -1.571 -1.571 ... -1.571 -1.571
    gamma_e             (idx) float64 -1.571 -1.571 -1.571 ... -1.571 -1.571
    latitude            (idx) float64 1.571 1.571 1.571 ... 1.571 1.571 1.571
    reward              (idx) float64 -76.5 -49.13 -70.51 ... -61.75 32.25
    reward_integral     (idx) float64 -255.3 -142.8 -225.7 ... -183.5 173.0
    delta_v             (idx) float64 -12.44 -10.66 -12.11 ... -11.75 -10.4 0.1
Attributes:
    env_surface_diameter:  1737400.0
    env_mu:                4904869500000.0
    env_stages:            (Stage(dry_mass=1, propellant_mass=0.1, specific_i...
    env_initial_latitude:  1.5707963267948966
    env_initial_altitude:  1000
    env_initial_theta_e:   1.5707963267948966

In [54]:
plt.figure()
smooth_signal(batch_results.reward_integral, wn=0.01).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Single simulation

In [66]:
simulation.restore_states()
with time_it("simulation run"):
    agent.epsilon = 0.0
    result = simulation.run(100)
    agent.epsilon = 0.1
result

simulation run: 0.024456384002405684 [s]


<xarray.Dataset>
Dimensions:             (d_2_0: 2, t: 100)
Coordinates:
  * t                   (t) float64 0.0 0.1 0.2 0.3 0.4 ... 9.5 9.6 9.7 9.8 9.9
Dimensions without coordinates: d_2_0
Data variables:
    command_engine_on   (t) bool True True True True ... True True True True
    command_drop_stage  (t) bool False False False False ... False False False
    gii                 (t, d_2_0) float64 -9.938e-17 -1.623 ... -1.623
    xii                 (t, d_2_0) float64 1.064e-10 1.738e+06 ... 1.738e+06
    vii                 (t, d_2_0) float64 0.0 0.0 7.073e-18 ... 2.153e-16 3.515
    aii                 (t, d_2_0) float64 -9.938e-17 -1.623 ... -1.623
    fii_thrust          (t, d_2_0) float64 0.0 0.0 2.082e-16 3.4 ... 0.0 0.0 0.0
    theta               (t) float64 1.571 1.571 1.571 ... 1.571 1.571 1.571
    theta_dot           (t) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    mass                (t) float64 1.02 1.02 1.019 ... 0.9998 0.9998 0.9998
    mass_dot            (t) float64 0.0 -0.003466 -0.003466 ... 0.0 0.0 0.0
    h                   (t) float64 1e+03 1e+03 1e+03 ... 1.057e+03 1.057e+03
    engine_on           (t) bool False True True True ... False False False
    stage_state         (t) int64 0 1 1 1 1 1 1 1 1 1 1 ... 2 2 2 2 2 2 2 2 2 2
    stage_idx           (t) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    gamma_i             (t) float64 0.0 1.571 1.571 1.571 ... 1.571 1.571 1.571
    gamma_e             (t) float64 0.0 1.571 1.571 1.571 ... 1.571 1.571 1.571
    latitude            (t) float64 1.571 1.571 1.571 ... 1.571 1.571 1.571
    reward              (t) float64 0.0 0.002997 0.02311 ... 56.13 56.5 56.86
    reward_integral     (t) float64 0.0 7.272e-06 0.00117 ... 228.9 234.5 240.2
    delta_v             (t) float64 0.0 8.625 18.03 29.42 ... 0.8349 0.9619 1.14
Attributes:
    env_surface_diameter:  1737400.0
    env_mu:                4904869500000.0
    env_stages:            (Stage(dry_mass=1, propellant_mass=0.02, specific_...
    env_initial_latitude:  1.5707963267948966
    env_initial_altitude:  1000
    env_initial_theta_e:   1.5707963267948966

In [67]:
plt.figure()
result.fii_thrust.plot.line(x="t")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
plt.figure()
result.mass.plot.line(x="t")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …